In [98]:
#install and import neccasary ....
!pip install googletrans==4.0.0-rc1
!pip install nltk

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import googletrans
from googletrans import Translator
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Buddhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Buddhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [107]:
#import dataset csv file.
df = pd.read_csv('dataset.csv')
df = df.iloc[100:200,2:4]
df

,Statement,Positive/Negative
100,අහින්සක ලමයි පවා යොදා ගනිමින් protest කරන්න්න ...,Negative
101,ස්ටලින් හිරකලාම GSP+ නැතිවෙන්න පුලුවන් කිවුව අ...,Negative
102,නරක් වෙච්ච කෑම කන්නේ නැ වගේ.. Expire වෙච්ච දේ...,Negative
103,NaN,NaN
104,NaN,NaN
...,...,...
195,ලංකාවේ බොහෝ දෙනෙක් දන්නේ නැ food intolerance ට...,Negative
196,මෙය මාගේ profile පින්තූරය පාවිච්චි කරමින් කරන්...,Negative
197,අන්න ඒකයි මේච්චර memes හැදෙන්නේ .ඉරිසියාව,Negative
198,ඔයාට වගේ ලස්සන වචන වලින් මගේ feeling එක expres...,Negative


In [108]:
#find whether is there enay null values
df.isnull().any()

Statement            True
Positive/Negative    True
dtype: bool

In [109]:
#drop null values
df.dropna(subset = ["Positive/Negative"], inplace=True)

In [110]:
#print present null values
print(df.isnull().sum())
print(df.shape)

Statement            0
Positive/Negative    0
dtype: int64
(92, 2)


In [111]:
#make numbers correctly
df = df.reset_index(drop=True)

In [112]:
#replace positive as 1 and negative aas 0
df.replace('Positive', 1, True)
df.replace('Negative', 0, True)
df

,Statement,Positive/Negative
0,අහින්සක ලමයි පවා යොදා ගනිමින් protest කරන්න්න ...,0
1,ස්ටලින් හිරකලාම GSP+ නැතිවෙන්න පුලුවන් කිවුව අ...,0
2,නරක් වෙච්ච කෑම කන්නේ නැ වගේ.. Expire වෙච්ච දේ...,0
3,ඉස්සරහ ගැන හිතනව නම් ඔයාටත් පුළුවන් target එක ...,1
4,ඔයාගේ ටාර්ගට් එක ඔය ඉන්න තැනට වඩා හොද තැනක් නම...,1
...,...,...
87,"මැච් එක දිනන්න බැරි වෙච්ච එක තමයි දුක, ඒකට ගො...",0
88,ලංකාවේ බොහෝ දෙනෙක් දන්නේ නැ food intolerance ට...,0
89,මෙය මාගේ profile පින්තූරය පාවිච්චි කරමින් කරන්...,0
90,අන්න ඒකයි මේච්චර memes හැදෙන්නේ .ඉරිසියාව,0


In [113]:
statement = df['Statement']
statement

0     අහින්සක ලමයි පවා යොදා ගනිමින් protest කරන්න්න ...
1     ස්ටලින් හිරකලාම GSP+ නැතිවෙන්න පුලුවන් කිවුව අ...
2     නරක් වෙච්ච කෑම කන්නේ නැ වගේ..  Expire වෙච්ච දේ...
3     ඉස්සරහ ගැන හිතනව නම් ඔයාටත් පුළුවන් target එක ...
4     ඔයාගේ ටාර්ගට් එක ඔය ඉන්න තැනට වඩා හොද තැනක් නම...
                            ...                        
87     මැච් එක දිනන්න බැරි වෙච්ච එක තමයි දුක, ඒකට ගො...
88    ලංකාවේ බොහෝ දෙනෙක් දන්නේ නැ food intolerance ට...
89    මෙය මාගේ profile පින්තූරය පාවිච්චි කරමින් කරන්...
90            අන්න ඒකයි මේච්චර memes හැදෙන්නේ .ඉරිසියාව
91    ඔයාට වගේ ලස්සන වචන වලින් මගේ feeling එක expres...
Name: Statement, Length: 92, dtype: object

In [114]:
#translate sinhala words into english word.
trans = Translator()
for i in range(len(statement)):
  t_statements = trans.translate(statement[i],dest="en") 
  statement[i] = t_statements.text
  print(i+1 , '=>', statement[i])

C:\Users\Buddhi\AppData\Local\Temp\ipykernel_6272\1407285937.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  statement[i] = t_statements.text


1 => Even the children of Ahinzi, you are on the Protest Karanna Highway. The country had this country from Arabah since the festival. I'm trying to forget the end of one and the internional festival exerment. You need the need to me than ever before. They do this to reduce the people coming to BUT to the Have to remember some. WE ALL ARE BALL LANKS .. 🤣
2 => The signs of Stalin is stalin, where you can not forget GSP +.
3 => Like not to eat spoil-eating.Watch in tube
4 => If you think about front, you can work for the target.When you used to think about the frame, how can you find that you should beat the game?
5 => If your targat is a better place than you are, your
"Plan right simple" 🙂
You have to jump out of the framework you are around.
6 => Get used to soaring like a lion
Even a shepherd can't control one lion ...
But one lion can take the whole of the lambs. The lion's mentality, the lamb, the shepherd, the mentality of the shepherd ...
Will win life
Status, which calls the Dat

31 => Hello, Kandy 14 to come back to that past
𝐋𝐈𝐌𝐈𝐓𝐄𝐃 𝐇𝐔𝐑𝐑𝐘 𝐇𝐔𝐑𝐑𝐘 𝐇𝐔𝐑𝐑𝐘 𝐇𝐔𝐑𝐑𝐘 𝐇𝐔𝐑𝐑𝐘 𝐇𝐔𝐑𝐑𝐘 ...
𝐎𝐧𝐥𝐢𝐧𝐞 𝐓𝐢𝐜𝐤𝐞𝐭𝐬 𝐓𝐢𝐜𝐤𝐞𝐭𝐬 𝐎𝐮𝐭 𝐘𝐨𝐮𝐫 𝐘𝐨𝐮𝐫 𝐘𝐨𝐮𝐫 𝐘𝐨𝐮𝐫 𝐆𝐫𝐚𝐛𝐬 ....
32 => Chatted at the Lines & Beyond Channel's Lines & Beyond's channel
33 => Happy loveless lovers' day, who celebrates true love, and the true values of love,
HAPPY VALENTINE'S Day Believers of Real Love ...
Love You All ....🙏🏻❤️
34 => Success, Luxury Life Style, Price Money, Attention, and Do You Dreaming to Get Stuff Like, and Love?If so stop chasing, then stop it today.All you have to do is become your best version.
35 => If you can get out of here, take a LUNCH Packet from here and give a help from him
Remember this page to help you to take his medication a few years ago?
36 => There are endemic personality, passion, Mindset & emotional balancing meethogs.If a man is unable to find his Passion & Personality, he or she will never receive its mental independence by 100%.You must have a clear understanding of who you are not only to

79 => In Sri Lanka, people are so unique to a child, as ugly?

Shame 😤

CLIMB HIGHER AND HIGHER LITTLE GIRL.Were All with you .. god Bless You more and more .. ❤️
80 => More than the government, these men work hard to bring dollars to the country.We are in a country that prefers.If there are corrupt rulers, how beautiful this country is.😔
81 => • Not easy to cry ...
82 => If you think like this, you'll feel Meems.Thanks everyone to get us to the odor.
Treat the peer that this is a false poster.
Konara brother was famous for me
83 => Does the time affect Match today?A lot of people asked.There's nothing better for scared
84 => Dubai, You Were So Good to Me!See You Soon! 🤘 Love 😘
85 => This is still a song that a lot of people ask!Thank you All ❤️ !!! Thank everyone who wished us !!!
86 => One of the most beautiful selfie seen today.Dasun Shanaka Breast greeting to Sri Lanka's cups !!!
87 => Vegetable 💪 Our Fort
88 => It is sad that the match could not win, so many UPSET, SORRY ABOUT IT.

In [115]:
# remove is,the,....
stopWords = set(stopwords.words('english'))
porterStemer = PorterStemmer()
text = [] 
for i in range(len(statement)):
#capital letters into simple letters
  txt = statement[i].lower()
#remove links
  txt = re.sub(r"http\S+|www\S+|https\S+", '',txt)
  txt = re.sub(r'\@\w+|\#',' ',txt)
#remove unwanted things ex:emojis
  txt = re.sub('[^a-z]', ' ', txt)
  txt_token = nltk.word_tokenize(txt)
  txt = [porterStemer.stem(word) for word in txt_token if word not in stopWords]
  txt = ' '.join(txt)
  text.append(txt)
  print( i+1 , '=>', txt)

1 => even children ahinzi protest karanna highway countri countri arabah sinc festiv tri forget end one internion festiv exer need need ever reduc peopl come rememb ball lank
2 => sign stalin stalin forget gsp
3 => like eat spoil eat watch tube
4 => think front work target use think frame find beat game
5 => targat better place plan right simpl jump framework around
6 => get use soar like lion even shepherd control one lion one lion take whole lamb lion mental lamb shepherd mental shepherd win life statu call data card call rs peopl ask flower flower life life maximum time peopl mother dude want walk around world complaint peopl thought start new thing peopl work dream guy futzeaqu know go away parent bm zero even fallen zero know brusley say someth knowledg give power charcter give inectct knowledg give power attitud give credit win
7 => whatev anyon say famili good friend world berit one love everyon
8 => dp educ lakh
9 => everybodi media one give trend topic everi day
10 => extent s

In [116]:
#sentences into single words
countVec = CountVectorizer()
x = countVec.fit_transform(text).toarray()
y = df['Positive/Negative']

In [126]:
# split dataset into train and tesst
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 1)

In [127]:
#add the model
classifier = GaussianNB()
classifier.fit(x_train, y_train)

GaussianNB()

In [128]:
y_predic = classifier.predict(x_test)

In [129]:
#get the performance
matrix = confusion_matrix(y_test, y_predic)
print(matrix)
print("performance","=",accuracy_score(y_test, y_predic)*100)

[[10  3]
 [ 4  2]]
performance = 63.1578947368421


In [130]:
# performance of the proposed model = 63%